In [1]:
library('tidyverse')
library('data.table')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.5
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [2]:
setwd('~/dpetrov/MarkB/Orchard2021Data/18_HAFPipe_Validation/Fastqs/')

In [3]:
files = list.files(pattern = '.gz')
data = data.frame()
for (f in files){
    name = strsplit(f, '_R')[[1]][1]
    df = cbind(name)
    data = rbind(data, df)
    
}

In [4]:
data = unique(data)

In [5]:
write.table(data, 'orch2021.samps.csv' , quote = FALSE, row.names = FALSE)

In [2]:
#rename filtered, sorted, deduped bam files

In [11]:
setwd('~/dpetrov/MarkB/Orchard2021Data/18_HAFPipe_Validation/Bams/')
files = list.files(pattern = '.bam')
files

[1] "205_S11_L001.filtered.sorted.deduped.bam" 
 [2] "206_S12_L001.filtered.sorted.deduped.bam" 
 [3] "207_S13_L001.filtered.sorted.deduped.bam" 
 [4] "208_S14_L001.filtered.sorted.deduped.bam" 
 [5] "209_S15_L001.filtered.sorted.deduped.bam" 
 [6] "210_S16_L001.filtered.sorted.deduped.bam" 
 [7] "211_S20_L001.filtered.sorted.deduped.bam" 
 [8] "212_S21_L001.filtered.sorted.deduped.bam" 
 [9] "213_S22_L001.filtered.sorted.deduped.bam" 
[10] "214_S23_L001.filtered.sorted.deduped.bam" 
[11] "215_S24_L001.filtered.sorted.deduped.bam" 
[12] "329_S32_L001.filtered.sorted.deduped.bam" 
[13] "330_S33_L001.filtered.sorted.deduped.bam" 
[14] "331_S327_L002.filtered.sorted.deduped.bam"
[15] "331_S34_L001.filtered.sorted.deduped.bam" 
[16] "332_S35_L001.filtered.sorted.deduped.bam" 
[17] "333_S36_L001.filtered.sorted.deduped.bam" 
[18] "334_S37_L001.filtered.sorted.deduped.bam" 
[19] "335_S331_L002.filtered.sorted.deduped.bam"
[20] "335_S38_L001.filtered.sorted.deduped.bam" 
[21] "336_S332_L002.filtered.sorted.deduped.bam"
[22] "336_S39_L001.filtered.sorted.deduped.bam" 
[23] "337_S40_L001.filtered.sorted.deduped.bam"

In [10]:
ids = read.csv('../SampleID_Meta.csv')
head(ids)

,Sample,Number,BamName,BaiName,DPName,AlignStats
,<fct>,<int>,<fct>,<fct>,<fct>,<fct>
1,tp3_F1_E1_rep,1,tp3_F1_E1_rep.bam,tp3_F1_E1_rep.bam.bai,tp3_F1_E1_rep.dp.txt,tp3_F1_E1_rep.AlignStats.txt
2,tp3_F1_E3_rep,3,tp3_F1_E3_rep.bam,tp3_F1_E3_rep.bam.bai,tp3_F1_E3_rep.dp.txt,tp3_F1_E3_rep.AlignStats.txt
3,tp3_F1_E4_rep,4,tp3_F1_E4_rep.bam,tp3_F1_E4_rep.bam.bai,tp3_F1_E4_rep.dp.txt,tp3_F1_E4_rep.AlignStats.txt
4,tp3_F1_E6_rep,5,tp3_F1_E6_rep.bam,tp3_F1_E6_rep.bam.bai,tp3_F1_E6_rep.dp.txt,tp3_F1_E6_rep.AlignStats.txt
5,tp3_F1_E7_rep,6,tp3_F1_E7_rep.bam,tp3_F1_E7_rep.bam.bai,tp3_F1_E7_rep.dp.txt,tp3_F1_E7_rep.AlignStats.txt
6,tp3_F1_E9_rep,8,tp3_F1_E9_rep.bam,tp3_F1_E9_rep.bam.bai,tp3_F1_E9_rep.dp.txt,tp3_F1_E9_rep.AlignStats.txt


In [21]:
for (f in files){
    num = strsplit(f, '_')[[1]][1]
    d.samp = ids %>% filter(Number == num)
    samp = as.character(d.samp %>% pull(Sample))
    new.name = paste0(samp, '.HiCov.bam')
    file.rename(from = f, to = new.name)
}


In [21]:
##assess coverage:
setwd('~/dpetrov/MarkB/Orchard2021Data/18_HAFPipe_Validation/Bams/')
files = list.files(pattern = 'MeanDP')
cov = data.frame()
for (f in files){
    df = fread(f)
    df.cov = cbind(f, df[1,1])
    cov = rbind(cov, df.cov)
}
names(cov) = c('file', 'coverage') 
cov = cov %>% rowwise() %>% mutate(amount.down = if_else(coverage > 100,
                                           (100/coverage),
                                           coverage))
write.csv(cov, './PreSubSamp.HiCovPoolSeqSamp.Coverage.csv', row.names = FALSE) 

In [19]:
(100/123)

[1] 0.8130081

In [20]:
cov

file,coverage,amount.down
<chr>,<dbl>,<dbl>
tp11_F1_E1.HiCov_MeanDP.txt,123.5450,0.8094217
tp11_F1_E3.HiCov_MeanDP.txt,123.8170,0.8076435
tp11_F1_E4.HiCov_MeanDP.txt,165.0190,0.6059908
tp11_F1_E5.HiCov_MeanDP.txt,154.5920,0.6468640
tp11_F1_E7.HiCov_MeanDP.txt,114.9830,0.8696938
tp11_F1_E8.HiCov_MeanDP.txt,103.1120,0.9698192
tp3_F1_E1.HiCov_MeanDP.txt,155.0910,0.6447827
tp3_F1_E10.HiCov_MeanDP.txt,102.4480,0.9761050
tp3_F1_E11.HiCov_MeanDP.txt,102.8350,0.9724316


In [19]:
1 - ((123.545/100) -1)

[1] 0.76455

In [2]:
##assess coverage:
setwd('~/dpetrov/MarkB/Orchard2021Data/18_HAFPipe_Validation/')
files = list.files(pattern = 'MeanDP')
cov = data.frame()
for (f in files){
    df = fread(f)
    df.cov = cbind(f, df[1,1])
    cov = rbind(cov, df.cov)
}
names(cov) = c('file', 'coverage') 
#cov = cov %>% rowwise() %>% mutate(amount.down = if_else(coverage > 100,
 #                                          1 - ((coverage/100) -1)
  #                                        , coverage))


In [4]:
cov %>% filter(coverage < 94)

file,coverage
<chr>,<dbl>
tp11_F1_E4.100x_MeanDP.txt,57.7341
tp11_F1_E5.100x_MeanDP.txt,70.2156
tp3_F1_E1.100x_MeanDP.txt,69.6543
tp3_F1_E2.100x_MeanDP.txt,82.1969
tp3_F1_E3.100x_MeanDP.txt,85.4757
tp3_F1_E4.100x_MeanDP.txt,80.9973
tp3_F1_E6.100x_MeanDP.txt,54.0684
tp3_F1_E7.100x_MeanDP.txt,81.3866


In [9]:
165.0190*(1-.34)

[1] 108.9125

In [7]:
pre.filt = read.csv("./Bams/PreSubSamp.HiCovPoolSeqSamp.Coverage.csv")
pre.filt

file,coverage,amount.down
<fct>,<dbl>,<dbl>
tp11_F1_E1.HiCov_MeanDP.txt,123.5450,0.76455
tp11_F1_E3.HiCov_MeanDP.txt,123.8170,0.76183
tp11_F1_E4.HiCov_MeanDP.txt,165.0190,0.34981
tp11_F1_E5.HiCov_MeanDP.txt,154.5920,0.45408
tp11_F1_E7.HiCov_MeanDP.txt,114.9830,0.85017
tp11_F1_E8.HiCov_MeanDP.txt,103.1120,0.96888
tp3_F1_E1.HiCov_MeanDP.txt,155.0910,0.44909
tp3_F1_E10.HiCov_MeanDP.txt,102.4480,0.97552
tp3_F1_E11.HiCov_MeanDP.txt,102.8350,0.97165
